# Tagging and Extraction Using OpenAI functions

In [6]:
import os

import openai
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

# Code of your application, which uses environment variables (e.g. from `os.environ` or
# `os.getenv`) as if they came from the actual environment.
# openai.api_key = os.environ['OPENAI_API_KEY']

True

In [71]:
# from typing import List
# from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from pydantic import BaseModel, Field

In [72]:
class Tagging(BaseModel):
    """Tag the piece of text with particular info."""

    sentiment: str = Field(
        description="sentiment of text, should be `pos`, `neg`, or `neutral`"
    )
    language: str = Field(description="language of text (should be ISO 639-1 code)")

In [73]:
convert_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Tag the piece of text with particular info.',
 'parameters': {'type': 'object',
  'properties': {'sentiment': {'description': 'sentiment of text, should be `pos`, `neg`, or `neutral`',
    'type': 'string'},
   'language': {'description': 'language of text (should be ISO 639-1 code)',
    'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [74]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [75]:
model = ChatOpenAI(temperature=0)

In [76]:
tagging_functions = [convert_to_openai_function(Tagging)]

In [77]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Think carefully, and then tag the text as instructed"),
        ("user", "{input}"),
    ]
)

In [78]:
model_with_functions = model.bind(
    functions=tagging_functions, function_call={"name": "Tagging"}
)

In [79]:
tagging_chain = prompt | model_with_functions

In [80]:
tagging_chain.invoke({"input": "I love langchain"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "pos",\n  "language": "en"\n}', 'name': 'Tagging'}})

In [81]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "sentiment": "neg",\n  "language": "it"\n}', 'name': 'Tagging'}})

In [82]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

In [83]:
tagging_chain = prompt | model_with_functions | JsonOutputFunctionsParser()

In [84]:
tagging_chain.invoke({"input": "non mi piace questo cibo"})

{'sentiment': 'neg', 'language': 'it'}

In [85]:
tagging_chain.invoke(
    {"input": "I have to go back to work tomorrow. School holidays are over."}
)

{'sentiment': 'neutral', 'language': 'en'}

In [86]:
tagging_chain.invoke(
    {"input": "I have to go back to work tomorrow. School holidays are too short."}
)

{'sentiment': 'neg', 'language': 'en'}

## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [87]:
from typing import Optional


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [88]:
class Information(BaseModel):
    """Information to extract."""

    people: list[Person] = Field(description="List of info about people")

In [89]:
convert_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'type': 'object',
  'properties': {'people': {'description': 'List of info about people',
    'type': 'array',
    'items': {'description': 'Information about a person.',
     'type': 'object',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'description': "person's age", 'type': 'integer'}},
     'required': ['name']}}},
  'required': ['people']}}

In [90]:
extraction_functions = [convert_to_openai_function(Information)]
extraction_model = model.bind(
    functions=extraction_functions, function_call={"name": "Information"}
)

In [91]:
extraction_model.invoke("Joe is 30, his mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha",\n      "age": 0\n    }\n  ]\n}', 'name': 'Information'}})

In [92]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Extract the relevant information, if not explicitly provided do not guess. Extract partial info",
        ),
        ("human", "{input}"),
    ]
)

In [93]:
extraction_chain = prompt | extraction_model

In [94]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "people": [\n    {\n      "name": "Joe",\n      "age": 30\n    },\n    {\n      "name": "Martha"\n    }\n  ]\n}', 'name': 'Information'}})

In [95]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

In [96]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]}

In [97]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [98]:
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")
)

In [99]:
extraction_chain.invoke({"input": "Joe is 30, his mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha'}]

## Doing it for real

We can apply tagging to a larger body of text.

For example, let's load this blog post and extract tag information from a sub-set of the text.

In [100]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()

In [101]:
len(documents)

1

In [102]:
doc = documents[0]

In [103]:
page_content = doc.page_content[:10000]

In [104]:
print(page_content[:1000])







LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [105]:
class Overview(BaseModel):
    """Overview of a section of text."""

    summary: str = Field(description="Provide a concise summary of the content.")
    language: str = Field(
        description="Provide the language that the content is written in."
    )
    keywords: str = Field(description="Provide keywords related to the content.")

In [106]:
overview_tagging_function = [convert_to_openai_function(Overview)]
tagging_model = model.bind(
    functions=overview_tagging_function, function_call={"name": "Overview"}
)
tagging_chain = prompt | tagging_model | JsonOutputFunctionsParser()

In [107]:
tagging_chain.invoke({"input": page_content})

{'summary': 'This article discusses the concept of building autonomous agents powered by LLM (large language model) as their core controller. It explores the key components of such agent systems, including planning, memory, and tool use. It also covers various techniques for task decomposition and self-reflection in autonomous agents. The article provides examples of case studies and challenges in implementing LLM-powered autonomous agents.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, task decomposition, self-reflection, case studies, challenges'}

In [120]:
from pydantic import BaseModel, Field


class Paper(BaseModel):
    """Information about papers mentioned."""

    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""

    papers: list[Paper]

In [121]:
convert_to_openai_function(Info)

{'name': 'Info',
 'description': 'Information to extract',
 'parameters': {'type': 'object',
  'properties': {'papers': {'type': 'array',
    'items': {'description': 'Information about papers mentioned.',
     'type': 'object',
     'properties': {'author': {'type': 'string'}},
     'required': ['title']}}},
  'required': ['papers']}}

### Manual fixed the wrong openai function

In [125]:
openai_function_manual = {
    "name": "Info",
    "description": "Information to extract",
    "parameters": {
        "type": "object",
        "properties": {
            "papers": {
                "type": "array",
                "items": {
                    "description": "Information about papers mentioned.",
                    "type": "object",
                    "properties": {
                        "title": {"type": "string"},
                        "author": {"type": "string"},
                    },
                    "required": ["title", "author"],
                },
            }
        },
        "required": ["papers"],
    },
}

In [126]:
# paper_extraction_function = [convert_to_openai_function(Info)]
paper_extraction_function = [openai_function_manual]
extraction_model = model.bind(
    functions=paper_extraction_function, function_call={"name": "Info"}
)
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [127]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': 'Wei et al.'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al.'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al.'}]

In [128]:
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text.

Make sure the title of paper is included."""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

In [129]:
extraction_chain = (
    prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")
)

In [130]:
extraction_chain.invoke({"input": page_content})

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': 'Wei et al.'},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': 'Liu et al.'},
 {'title': 'ReAct (Yao et al. 2023)', 'author': 'Yao et al.'},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight (CoH; Liu et al. 2023)',
  'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation (AD; Laskin et al. 2023)',
  'author': 'Laskin et al.'}]

In [131]:
extraction_chain.invoke({"input": "hi"})

[]

In [132]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

In [133]:
splits = text_splitter.split_text(doc.page_content)

In [134]:
len(splits)

14

In [135]:
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [136]:
flatten([[1, 2], [3, 4]])

[1, 2, 3, 4]

In [137]:
print(splits[0])

LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general probl

In [138]:
from langchain.schema.runnable import RunnableLambda

In [139]:
prep = RunnableLambda(lambda x: [{"input": doc} for doc in text_splitter.split_text(x)])

In [140]:
prep.invoke("hi")

[{'input': 'hi'}]

In [141]:
chain = prep | extraction_chain.map() | flatten

In [142]:
chain.invoke(doc.page_content)

[{'title': 'Chain of thought', 'author': 'Wei et al. 2022'},
 {'title': 'Tree of Thoughts', 'author': 'Yao et al. 2023'},
 {'title': 'LLM+P', 'author': 'Liu et al. 2023'},
 {'title': 'ReAct', 'author': 'Yao et al. 2023'},
 {'title': 'Reflexion', 'author': 'Shinn & Labash 2023'},
 {'title': 'Reflexion: A Framework for Self-Reflection in Reinforcement Learning',
  'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight: Improving Language Models with Sequential Feedback',
  'author': 'Liu et al.'},
 {'title': 'Algorithm Distillation: Learning Process of Reinforcement Learning',
  'author': 'Laskin et al.'},
 {'title': 'Algorithm Distillation: A Unified Perspective on Model Compression, Transfer, and Multitask Learning',
  'author': 'Laskin et al.'},
 {'title': 'RL^2: Fast Reinforcement Learning via Slow Reinforcement Learning',
  'author': 'Duan et al.'},
 {'title': 'LSH (Locality-Sensitive Hashing)', 'author': ''},
 {'title': 'ANNOY (Approximate Nearest Neighbors Oh Yeah)', 'author'